<img style="float: left;;" src='Figures/alinco.png' /></a>

# <center> <font color= #000047> Módulo 1: Teoría de Redes Neuronales Convolucionales



<div class="alert alert-warning">
En este cuaderno nos familiarizaremos con un tipo de *capa* para redes neuronales artificiales llamadas capas convolucionales. Los datos que intentaremos modelar utilizando este tipo de redes serán imágenes.
</div>


## Imagenes

Para una computadora, una imagen es una matriz de datos, donde cada píxel está representado por uno o más valores:


### Matriz con un valor por píxel = imágenes en escala de grises

<img src="Figures/lincoln_pixel_values.png" alt="Grayscale Image" width="700">

### Matriz con tres valores por píxel = imágenes en color
 <img src="Figures/color_images.png" alt="Decomposition of a color image" width="400">


## Convolución

Antes de meternos de lleno en las redes, necesitamos comprender bien el concepto de **convolución**. La convolución es un operador matemático que se define como la integral del producto de dos funciones donde una de ellas está desplazada una distancia $t$.

$$ (f \ast g)(t) = \int_{-\infty}^{\infty} f(x)g(t-x) dx  $$

Nosotros vamos a adaptar este operador a una versión bidimensional y discreta. Esta sería la versión bidimensional:

$$ (f \ast g)(i,j) = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} f(x,y)g(i-x,j-y) dx  $$

Y esta la versión bidimensional y discreta con paso 1, que es la que nos interesa:

$$ (f \ast g)(i,j) = \sum_{-\infty}^{\infty} \sum_{-\infty}^{\infty} f(x,y)g(i-x,j-y)$$


**¿Y esto para qué nos va a servir?**

Nuestro cerebro integraba simples estímulos visuales procedentes de cada fotorreceptor de la retina para producir elementos de información cada vez más compleja y elaborada para permitir luego su reconocimiento. Es como decir que para reconocer una cara nuestro sistema visual registra primero fragmentos de la imagen como pupilas, comisuras de labios, lóbulos de orejas… para luego formar ojos, bocas, orejas… para, finalmente, formar caras. Bueno, pues con la operación de convolución vamos a hacer algo así.


Partamos de una imagen cualquiera, tomémosla en escala de grises para que sea aún más sencilla. Por ahora solo tenemos pixeles.

- ¿Cuáles serían las características más sencillas que podríamos encontrar? 

Quizá serían las características que encontraríamos en regiones de tamaño $3 \times 3$ de la imagen. ¿Qué cabe en una región tan pequeña? Podríamos encontrar un borde vertical, un borde horizontal, una esquina, un punto… cosas así. 

Echemos un vistazo de nuevo a la expresión de la convolución en su versión bidimensional  y discreta y hagámosle unas adaptaciones:

En primer lugar, démosle sentido a $g$. Esta función va a ser nuestra imagen. La función $f$ va a ser otra imagen, pero en este caso, de tamaño $3 \times 3$. Además le vamos a cambiar el nombre y, a partir de ahora, la llamaremos **kernel**. Le asignaremos estos valores: 

\begin{bmatrix}
 1 & 0 & -1\\ 
 1 & 0 & -1\\ 
 1& 0 & -1
\end{bmatrix}

Puesto que el kernel es de tamaño $3 \times 3$ los índices de las sumatorias serán:

$$ conv2D(i,j) = \sum_{y=0}^{2} \sum_{x=0}^{2} kernel(x,y) \cdot imagen(i-x,j-y)$$

Y para hacerlo un poco más interpretable, simplemente vamos a cambiar la manera en que superponemos el kernel sobre la imagen cambiando restas por sumas:

$$ conv2D(i,j) = \sum_{y=0}^{2} \sum_{x=0}^{2} kernel(x,y) \cdot imagen(i+x,j+y)$$

Vamos a visualizarlo con un ejemplo. Nuestra imagen es de $6 \times 6$ y tiene los valores que representa la figura.


<img src="Figures/Kernel.svg" width="50%">

Queremos calcular el valor de la convolución en las coordenadas (2,1). Gráficamente se vería como la superposición del kernel sobre la imagen en esas coordendas y multiplicar celda a celda sus correspondientes valores para, finalmente, sumarlo todo. El resultado que nos devuelve es 765. Es decir, un valor alto. 

- ¿Qué pasaría si el área de $3 \times 3$ de la imagen sobre la que superponemos el kernel fuera totalmente homogénea? 

Efectivamente, el resultado sería 0. Entonces así ya tenemos un detector de bordes verticales.

<img src="Figures/Kernel2.svg" width="30%">

<div class="alert alert-warning">
    
**La operación de convolución puede ser implementada por una neurona!**

i.e., Los valores del kernel son equivalentes a los pesos de una neurona, la zona de la imagen donde se superpone el kernel son las entradas a la neurona y la sumatoria de la convolución se ejecuta de la misma forma que el sumatorio de la neurona.

Todo esto nos lleva a crear nuevos tipos de capas en las redes: las **capas convolutivas**.
</div>

## Capas convolutivas

Las capas de red que hemos visto hasta ahora se denominan *fully connected (fc)* o **totalmente conectadas**. Esto significa que todas las salidas de una capa anterior están conectadas a todas y cada una de las entradas de la capa siguiente. Por ejemplo, si hubiera 25 neuronas en la capa anterior, cada neurona de la capa siguiente tendría 25 entradas (y una más para el *bias*). 

>En las **capas convolutivas** esto ya no es así. Cada neurona de una capa convolutiva está conectada solo a un conjunto de salidas de la capa anterior. Además, las capas convolutivas esperan una disposición bidimensional de las entradas, no lineal. La figura muestra una neurona (esfera azul) con nueve entradas conectada a un grupo local de nueve valores. El cubo azul representa su salida.

<img src="Figures/NeuronaConv.jpg" width="40%">


Cada neurona de una capa convolutiva comparte el mismo conjunto de pesos, por lo que se podría decir que las neuronas de la capa convolutiva se replican de forma matricial a lo largo y ancho de la entrada. En la figura siguiente vemos la conexión de todas las neuronas de la capa convolutiva.

<img src="Figures/CapaCompleta.jpg" width="40%">

Vamos a echar cuentas, fíjate que hay $10\times15 = 150$ entradas distintas, cada neurona tiene un **campo receptivo** de $3\times3=9$ entradas. Si las alineamos a lo largo y ancho para que se cubran todas las entradas tenemos que hay $8\times13=104$ neuronas. ¿Cuántos pesos distintos hay? ¿$104\times9$? ¡¡Nooo!! Solo $9$. Cada neurona comparte el mismo conjunto de pesos, aunque los valores de las entradas son, obviamente, distintos.

Podemos hacer un detector de bordes horizontales, verticales, diagonales, detectores de esquinas, puntos, etc. en función del **kernel** que definamos. Cada “pixel” de esta nueva "imagen" ya no representa  un simple valor de iluminación aislado, representa una pequeña **característica** local del objeto que contenga. Pero con una sola característica no vamos a ir muy lejos, deberíamos tener más. Supongamos que elegimos bordes horizontales, verticales, diagonales hacia la derecha y diagonales hacia la izquierda. Al aplicar estos cuatro filtros, *kernels* o conjuntos de pesos obtendremos cuatro “imágenes” de características nuevas que, convenientemente apiladas, formarán un **tensor** de $4\times8\times13$. Cada arreglo bidimensional de neuronas (representados con un color diferente) posee un conjunto de pesos único y compartido entre todas sus neuronas. Cada arreglo genera un **canal** diferente en la salida (representado por los cubos azules, verdes, rosas y amarillos).

<img src="Figures/4canales.jpg" width="50%">

\- *¿Tensor? ¿Qué es un tensor?* - En informática se suele llamar a un tensor como un arreglo bidimensional de objetos "*array* bidimensional" o matriz. Cuando el arreglo es tridimensional lo solemos llamar "*array* tridimensional", pero no deberíamos llamarlo matriz tridimensional (eso es una expresión impropia), sino **tensor**. En el argot matemático, físico o ingenieril, cualquier arreglo de valores de tres dimensiones o más se denomina **tensor**.


Tenemos por tanto este tensor formado por diferentes características de la imagen. ¿Podríamos repetir de nuevo este proceso, pero ahora sobre este tensor? Rápidamente nos surgen un par de preguntas: ¿Con qué pesos? ¿Cómo se conectarían ahora las neuronas?  Antes, la imagen de partida era bidimensional, ahora tenemos un tensor tridimensional. 

No olvidemos que estamos utilizando neuronas, y como tales, pueden aprender por ellas mismas su conjunto de pesos, solo se necesita descender por el gradiente de una función de pérdida (ya veremos luego cómo crear esta función). Por tanto, esos pesos que habíamos propuesto para detectar bordes y demás los vamos a dejar libres (“aprendibles”). Esto significa que será la propia red la que se encargue de ver si efectivamente son bordes u otras características las que realmente necesita obtener.

En cuanto a la conexión, será muy parecido a lo anterior. Cada neurona se conectará a una región local en cuanto a la dimensión $x$ e $y$, y a todos los valores del tensor  en la dimensión $z$. Como ilustra la figura, la primera neurona estará conectada a su correspondiente campo receptivo de $4\times3\times3$. Por lo tanto, cada neurona tendrá 36 entradas.

<img src="Figures/9x4cr.jpg" width="40%">

Es muy importante ahora reflexionar sobre lo que contiene la salida de la neurona de la ilustración anterior. Es una combinación lineal de 36 valores, pero cada uno de esos valores es, a su vez, otra combinación lineal de valores anteriores correspondientes a la imagen original y cuyo valor dependía de haber encontrado una determinada característica simple o no. Por tanto, el valor de esta neurona representa haber encontrado una característica más compleja. Quizá una combinación de un borde con una esquina, o un borde más largo… Su campo receptivo sobre la imagen original es de $5\times5$. La imagen siguiente ilustra simplificadamente por qué la última salida depende, en última instancia, de 5 valores de la entrada original.

<img src="Figures/camporeceptivo.svg" width="30%">

De la misma forma que antes, añadimos más canales y obtenemos un nuevo tensor de salida. Si añadimos 8 canales obtendremos un tensor de dimensión 8 en el eje $z$. Igual que antes, si el tensor de entrada tiene como ancho y alto $(m,n)$ el tensor de salida tendrá unas dimensiones $x$ e $y$ de $(m-2,n-2)$.

<img src="Figures/tensor8x6x11.jpg" width="60%">

## Clasificación del MNIST con una red convolutiva

## Data Pre-Processing

Antes de empezar con la clasificación debemos de hacer el preprocesado

Lo primero que haremos es crear una función que realize la normalización de los datosde [0,255] to [0,1]:


### Implementación de una función de normalización
<div class="alert alert-success">
Implementar una función que normalice los datos.
<ul>
  <li>Inputs enteros en el inetrvalo [0,255]</li>
  <li>Outputs valores flotantes en el intervalo [0,1]</li>
</ul>
</div>

### Expander la dimensión de la entrada

Cuando cargamos el conjunto de datos MNIST, cada dígito estaba representado por una matriz de tamaño $(28, 28)$. Sin embargo, la red neuronal artificial que construiremos utiliza el concepto de canales de color y mapas de características incluso para imágenes en escala de grises. Esto significa que tenemos que transformar $(28, 28)$ a $(28, 28, 1)$.

<div class="alert alert-success">
Escriba un fragmento de código que agregue una nueva dimensión a `x_train` y `x_test`.
<ul>
  <li>El tamaño de `x_train` debe ser $(60000, 28, 28, 1)$</li>
  <li>El tamaño de `x_test`  debe ser $(10000, 28, 28, 1)$</li>
</ul>

</div>

In [ ]:
y_train

## Target Pre-Processing

Para clasificar nuestros dígitos, necesitamos usar una codificación one-hot para representar los resultados de destino. La codificación one-hot es una solución robusta pero simple para representar objetivos multicategóricos.

Esta codificación es una representación ideal para entrenar un modelo usando un algoritmo de descenso de gradiente con la [función softmax](http://www.cs.toronto.edu/~guerzhoy/321/lec/W04/onehot.pdf) que discutimos en un cuaderno anterior.

### Example of one-hot encoding

Ejemplo:

$$
\begin{equation*}
\mathbf{y} =
\left[ \begin{array}{c} 2 \\ 8 \\ 0 \\ 6 \\ \vdots\end{array} \right]
\Longrightarrow
\begin{bmatrix}
  0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
  1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
  0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\
  \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots
\end{bmatrix}
\end{equation*}
$$

En el lado izquierdo tenemos un vector de etiquetas objetivo $\mathbf{y}$ con $K=10$ número de clases. En el lado derecho podemos ver la versión codificada one-hot del vector $\mathbf{y}$ donde cada elemento $\in \mathbf{y}$ ha sido transformado en un vector fila de $K$-dimensional. Solo el elemento $\mathbf{y}_i$ se ha establecido en 1, el resto son 0. Por ejemplo, el primer elemento de $\mathbf{y}$ es 2, lo que significa que el vector codificado one-hot será todo ceros excepto para la posición 2 (0-indexación). De manera similar, el tercer elemento de $\mathbf{y}$ es 0, lo que significa que el vector codificado one-hot será todo ceros excepto la posición 0.

La idea central es que transforme datos multicategóricos en una combinación de varias clases individuales. Al hacer esto podemos, para cada ejemplo, ver si pertenece a alguna clase, donde 1 indica que sí y 0 en caso contrario.


### Implement a Function for One-Hot Encoding

<div class="alert alert-success">
Implemente una función que codifique eone-hot encodign para $K$ clases:
<ul>
  <li>El primer argumento es un vector con $N$ muestras (dimensiones)</li>
  <li>El segundo argumento es un número $K$ que significa el número de clases</li>
  <li>Para cada muestra del vector, creará una matriz con dimensiones $K$</li>
  <li>La matriz codificada one-hot debe tener ceros en todas las posiciones excepto en la posición indicada por la muestra actual en el vector de entrada</li>
</ul>
</div>



In [ ]:
#y_train = keras.utils.to_categorical(y_train, 10)
#y_test = keras.utils.to_categorical(y_test, 10)


Ahora que hemos agregado una dimensión adicional a los datos de entrada y que hemos codificado los valores objetivo en caliente, echemos un vistazo a las formas de las matrices de datos.

Vamos a componer una red convolutiva para clasificar el conjunto MNIST. Tenemos como entrada una imagen de $28\times28$ pixels en escala de grises. Por tanto, esto es un tensor de $1\times28\times28$. Pasaremos esta entrada por una primera capa convolutiva de 32 filtros, lo cual nos devolverá un tensor de $32\times26\times26$. De nuevo, pasaremos este tensor ahora como entrada a una segunda capa convolutiva con 64 filtros. ¿Por qué 32 y 64 filtros? Bueno, te habrás imaginado ya que son hiperparámetros. El resultado es un tensor de $64\times24\times24$. Lo que estamos haciendo con las sucesivas capas convolutivas es ir extrayendo características cada vez más complejas que faciliten luego la labor de clasifición.

Llegados a este punto, aplanaremos (*flatten*) este tensor a fin de convertirlo en un vector con una dimensión de $64\times24\times24 = 36864$. Ahora podremos alimentar a una capa *fully connected* (FC) de 128 neuronas. Su resultado lo pasaremos finalmente por una última capa *fully connected* de 10 neuronas.
 

<img src="Figures/modelConv.svg" width="80%">

Esta es una aproximación correcta a una red convolutiva, pero no es la práctica habitual. Hay un problema de tamaño. Si nos fijamos en la capa *fully connected* de 128 neuronas veremos que cada neurona tiene 36.864 entradas (1 más si contamos el bias). Lo que hace un total de ¡¡4.718.720 pesos para esa capa!! Para aliviar este enorme flujo de datos se emplean diferentes técnicas. Una de las más comunes es el *maxpolling*.

### Maxpooling

Si interpretamos con detenimiento la información que contiene un tensor, podremos eliminar gran cantidad de datos sin socavar una parte significativa de esta información. Escojamos, por ejemplo, el tensor con dimensión $32 \times 26 \times 26$. El valor de cada celda de este tensor corresponde a haber encontrado una determinada característica en una región concreta de la imagen original. Un valor alto significa que esa característica se encontró muy claramente. Un valor bajo o cercano al cero indica que esa característica no está presente en la correspondiente zona de la imagen. También podríamos decir que encontrar esa característica una celda a la derecha, a la izquierda, arriba o abajo no debería representar mayor inconveniente. *Maxpooling* aprovecha este hecho eliminando el 75% de los datos sin reducir excesivamente la información que contiene el tensor. Para ello se toma cada canal del tensor (es decir, se divide a lo largo del eje $z$) y se agrupan en regiones de $2 \times 2$. De cada grupo se extrae la celda que mayor valor tenga. Al final de este proceso, tendremos un tensor con las dimensiones $x$ e $y$ reducidas a la mitad.

<img src="Figures/MNISTconvolutional-maxpooling.svg" width="100%">


### Dropout

Las redes convolutivas, como todas las demás redes, pueden adolecer de problemas de sobreajuste (*overfitting*). Para solventar esta contrariedad existen varias técnicas que reducen su efecto. Una de las más usadas y efectivas es el <i>**dropout**</i>. Esta técnica consiste en deshabilitar temporalmente un porcentaje aleatorio de las neuronas de una capa. Es decir, durante un cierto tiempo estas neuronas están, pero como si no estuvieran. El conjunto de neuronas que se habilitan y deshabilitan se van alternando aleatoriamente durante el proceso de entrenamiento.  Con esto se consigue que todas las neuronas deban emplearse a fondo para lograr un buen nivel de *accuracy*.

<img src="Figures/dropout.jpg" width="50%">



Por supuesto, el proceso de *dropout* solo se realiza durante el entrenamiento, no en producción.

### ReLU

Hemos visto que a la salida de las neuronas se les aplica una función de activación. Hasta ahora, hemos visto las funciones sigmoide y softmax, pero existen más. Una de ellas es la función **ReLU** (*Rectified Linear Unit*), cuya definición es:

$$ReLU(x) = max(0, x)$$

Lo cual significa que si la entrada es un valor mayor que 0, la salida será ese mismo valor. Pero si la entrada es negativa la salida será 0. Esta es una función predilecta en la composición de redes convolutivas ya que ofrece muy buenos resultados empíricos. Pero, ¿por qué funciona mejor que la función sigmoide? Si observamos el comportamiento de la función sigmoide con valores lejanos al 0 vemos que es una curva casi plana. Esto significa que su derivada en esos puntos es prácticamente 0, con lo que el descenso por el gradiente en esas zonas sería muy lento y, por tanto, el aprendizaje también . Este problema no lo tiene la función ReLU, su derivada en el semieje positivo es siempre 1.  Pero, ¿y qué ocurre con el semieje negativo? Bueno, cuando la salida de la neurona es negativa su paso por la función ReLU la hace 0. Asumimos que la inicialización aleatoria de los pesos hace que, dada una entrada, parte de las neuronas de una capa obtengan salidas negativas y la otra parte positivas. Esto hace que solo las neuronas con salida positiva intervengan en el proceso de representación del resultado, lo cual reduce su complejidad.




### Modelo y entrenamiento

A diferencia del modelo básico anterior que utilzamos para clasificar el *dataset* MNIST, este modelo incluirá capas convolutivas <code>Conv2D</code>, dropout <code>Dropout</code> y funciones de activación ReLU <code>activation='relu'</code>.

# Hiperparametros del entrenamiento


# Entrenar la red

In [ ]:
# Code to plot the 5th training sample.


**Ejercicio:**

Modifica algunos hiperparámetros y observa si hay cambios significativos.
>Por ejemplo, cambia la función ReLU por una sigmoide

>Adadelta por SGD, añade más capas convolutivas o cambia el tamaño del kernel a  5×5 .